# Similarity-Based Constraint Score For Feature Selection Notebook

In this notebook, you will find a comprehensive summary of the project, detailing key steps undertaken and presenting results obtained during the implementation of our innovative constraint score.



In [7]:
%cd /Users/fabiogadegbeku/Desktop/M1_DS/projet_recherche/Similarity-Based-CS
import numpy as np
import pandas as pd
import sklearn.datasets as skd

/Users/fabiogadegbeku/Desktop/M1_DS/projet_recherche/Similarity-Based-CS/.venv/lib/python3.11/site-packages/IPython/core/magics/osm.py:417: UserWarning: using dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


/Users/fabiogadegbeku/Desktop/M1_DS/projet_recherche/Similarity-Based-CS


In [18]:
from c_scores import laplacian_score
X = skd.load_wine()
X_wine = X.data
laplacian_score(X_wine)

array([4.69962897e-02, 5.04996564e-02, 2.60937623e-02, 8.23879288e-03,
       9.64238725e-04, 2.53528247e-02, 1.31247495e-02, 5.38465218e-02,
       4.48234774e-02, 1.61773975e-02, 2.35186716e-02, 2.60547481e-02,
       2.09771285e-06])